---
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.17.1
  kernelspec:
    display_name: Big-G Express
    language: python
    name: big-g
---

In [2]:
import pandas as pd
import geopandas as gpd
from datetime import datetime

In [3]:
faults_raw = pd.read_csv("../data/J1939Faults.csv", dtype={"EquipmentID": str, 'spn':int})
diagnostics_raw = pd.read_csv("../data/vehiclediagnosticonboarddata.csv")

In [4]:
# prepare faults
faults_drop_cols = ["actionDescription", "activeTransitionCount", "eventDescription", "ecuSource", "ecuSoftwareVersion", "ecuModel", "ecuMake", "faultValue", "MCTNumber", "LocationTimeStamp"]
faults = faults_raw.drop(columns=faults_drop_cols)
print("\n\n--------SHAPE OF FAULTS--------")
display(faults.shape)



--------SHAPE OF FAULTS--------


(1187335, 10)

In [5]:
# join diagnostics
print("--------NaNs--------")
print(diagnostics_raw.isna().sum())
n_ids = len(diagnostics_raw['Id'])
n_unique_id = diagnostics_raw['Id'].nunique()
n_un_faults = diagnostics_raw['FaultId'].nunique()
diagnostics_raw["Value"] = diagnostics_raw["Value"].replace(
    {"FALSE": False, "TRUE": True}
)

# pivot diagnostics to long format
diagnostics = diagnostics_raw.pivot(
    index="FaultId", columns="Name", values="Value"
)

print(f"\nlen(Id): {n_ids}", f"\nN unique_Id: {n_unique_id}")
print("\n--------RECORD ID vs FAULT ID--------")
print(f"n_unique FaultID: {n_un_faults}", f"\nn_unique RecordID: {faults['RecordID'].nunique()}")
joined = faults.merge(diagnostics, how = "inner", left_on='RecordID', right_on='FaultId')

--------NaNs--------
Id         0
Name       0
Value      0
FaultId    0
dtype: int64

len(Id): 12821626 
N unique_Id: 12821626

--------RECORD ID vs FAULT ID--------
n_unique FaultID: 1187335 
n_unique RecordID: 1187335


In [6]:
joined_pre_station_filter = joined
# filter out near service stations
joined_pre_station_filter = joined
print("Labeling faults near service stations...")
stations = pd.DataFrame(
    {
        "lat": [36.0666667, 35.5883333, 36.1950],
        "lon": [-86.4347222, -86.4438888, -83.174722],
    }
)
threshold_miles = 0.5
threshold_meters = threshold_miles * 1609.34
# create geodataframes with geopandas
gdf_joined = gpd.GeoDataFrame(
    joined,
    geometry=gpd.points_from_xy(joined.Latitude, joined.Longitude),
    crs="EPSG:4326",  # WGS84 coord ref sys (lat/lon)
)
gdf_stations = gpd.GeoDataFrame(
    stations,
    geometry=gpd.points_from_xy(stations.lat, stations.lon),
    crs="EPSG:4326",
)
target_crs = "EPSG:9311"
# reproject onto new crs for better distance measurement
gdf_joined_proj = gdf_joined.to_crs(target_crs)
gdf_stations_proj = gdf_stations.to_crs(target_crs)
# create buffers around stations
station_buf = gdf_stations_proj.geometry.buffer(threshold_meters)
combined_buffer = (
    station_buf.union_all()
)  # turns into single geometry which helps with efficiency
is_within = gdf_joined_proj.geometry.within(combined_buffer)
joined["nearStation"] = is_within.values
joined_post_filter = joined[~joined["nearStation"]]
print("\nDone! \nFaults within 1km of service station labeled in 'joined'.")
print(
    f"When filtered, this removes {len(joined_pre_station_filter['RecordID']) - len(joined_post_filter['RecordID'])} rows"
)
# filter out active=False
joined_active = joined[joined["active"]]
joined = joined_active
print(
    f"\nNumber of rows after filtering active=False out: {len(joined_active['active'])}"
)
print(
    f"Rows removed: {len(joined_pre_station_filter['RecordID']) - len(joined_active['active'])}"
)

Labeling faults near service stations...

Done! 
Faults within 1km of service station labeled in 'joined'.
When filtered, this removes 129284 rows

Number of rows after filtering active=False out: 608454
Rows removed: 578881


In [7]:
target_spn = 5246

# Ensure EventTimeStamp is datetime
joined["EventTimeStamp"] = pd.to_datetime(joined["EventTimeStamp"])

# --- SORTING STEP ---
# Sort by EquipmentID and then chronologically by EventTimeStamp
print("Sorting data by EquipmentID and EventTimeStamp...")
joined = joined.sort_values(by=["EquipmentID", "EventTimeStamp"]).copy()
print("Sorting complete.")

# Create a Series containing only the timestamps of trigger events
trigger_timestamps_only = joined["EventTimeStamp"].where(joined["spn"] == target_spn)

# For each row, find the timestamp of the *next* trigger event within its group
# Group by EquipmentID and use backward fill (bfill)
# This fills NaT values with the next valid timestamp in the group
print("Calculating next trigger time...")
joined["next_trigger_time"] = trigger_timestamps_only.groupby(
    joined["EquipmentID"]
).bfill()

# Calculate the start of the 2-hour window before the next trigger
joined["window_start_time"] = joined["next_trigger_time"] - pd.Timedelta(hours=2.0)

# Label rows as True if their timestamp falls within the window:
#    [wind`ow_start_time, next_trigger_time]
#    Also ensure that a next_trigger_time actually exists (it's not NaT)
print("Labeling derate window...")
joined["derate_window"] = (
    (joined["EventTimeStamp"] >= joined["window_start_time"])
    & (joined["EventTimeStamp"] <= joined["next_trigger_time"])
    & (joined["next_trigger_time"].notna())
)

# --- Verification ---
print("\nVerification:")
print(
    "Value counts for 'derate_window':\n",
    joined["derate_window"].value_counts(),
)
print(
    "\nValue counts for 'spn' (to confirm target SPN exists):\n",
    joined["spn"].value_counts(),
)

# Display some rows where derate_window is True (if any)
print("\nSample rows where derate_window is True:")
print(
    joined[joined["derate_window"]][
        ["EquipmentID", "EventTimeStamp", "spn", "next_trigger_time"]
    ].head()
)

# Display some rows around a trigger event for a specific EquipmentID
# Find an EquipmentID that has a trigger event
example_eqid = joined.loc[joined["spn"] == target_spn, "EquipmentID"].iloc[0]
print(f"\nSample data around trigger for EquipmentID: {example_eqid}")
example_trigger_time = joined.loc[
    (joined["EquipmentID"] == example_eqid) & (joined["spn"] == target_spn),
    "EventTimeStamp",
].min()
# Filter data around that time for that equipment
print(
    joined[
        (joined["EquipmentID"] == example_eqid)
        & (joined["EventTimeStamp"] >= (example_trigger_time - pd.Timedelta(hours=3)))
        & (joined["EventTimeStamp"] <= (example_trigger_time + pd.Timedelta(hours=1)))
    ][
        [
            "EquipmentID",
            "EventTimeStamp",
            "spn",
            "next_trigger_time",
            "window_start_time",
            "derate_window",
        ]
    ]
)

joined = joined.drop(columns=['next_trigger_time', 'window_start_time', 'CruiseControlActive', 'AcceleratorPedal', 'DistanceLtd', 'FuelLevel', 'ParkingBrake', 'SwitchedBatteryVoltage', 'RecordID', 'ESS_Id', 'ecuSerialNumber', 'CruiseControlSetSpeed', "IgnStatus", 'LampStatus',    "IntakeManifoldTemperature",
    "ServiceDistance",])

/var/folders/1c/yvl84p5d2rv0n8dkqqjgmkdm0000gn/T/ipykernel_26516/1830182156.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined["EventTimeStamp"] = pd.to_datetime(joined["EventTimeStamp"])


Sorting data by EquipmentID and EventTimeStamp...
Sorting complete.
Calculating next trigger time...
Labeling derate window...

Verification:
Value counts for 'derate_window':
 derate_window
False    607254
True       1200
Name: count, dtype: int64

Value counts for 'spn' (to confirm target SPN exists):
 spn
111      183752
929      128777
96        48062
829       46179
639       20640
          ...  
703           1
707           1
5395          1
32000         1
1265          1
Name: count, Length: 450, dtype: int64

Sample rows where derate_window is True:
       EquipmentID      EventTimeStamp   spn   next_trigger_time
996835   105349576 2018-07-06 09:42:48  5246 2018-07-06 09:42:48
156408   105360462 2015-08-10 10:26:40  1569 2015-08-10 10:26:40
156409   105360462 2015-08-10 10:26:40  3360 2015-08-10 10:26:40
156410   105360462 2015-08-10 10:26:40  5246 2015-08-10 10:26:40
185581   105411041 2015-08-31 13:50:42  5246 2015-08-31 13:50:42

Sample data around trigger for EquipmentID

In [8]:
# some feature engineering:
joined['time_since_last_fault'] = joined.groupby('EquipmentID')['EventTimeStamp'].diff().dt.total_seconds().astype(float)
print(joined['time_since_last_fault'])
joined['fault_frequency'] = joined.groupby('EquipmentID')['spn'].transform('count')

1008925            NaN
985277             NaN
985237          3221.0
985170          5617.0
985179     233898893.0
              ...     
1157136          478.0
4245               NaN
4427            6371.0
6438          164454.0
4952               NaN
Name: time_since_last_fault, Length: 608454, dtype: float64


In [9]:
col_order = [
    "EquipmentID",
    "EventTimeStamp",
    "spn",
    "fmi",
    "active",
    "derate_window",
    "time_since_last_fault",
    "fault_frequency",
    "Latitude",
    "Longitude",
    "nearStation",
    "Speed",
    "BarometricPressure",
    "EngineCoolantTemperature",
    "EngineLoad",
    "EngineOilPressure",
    "EngineOilTemperature",
    "EngineRpm",
    "EngineTimeLtd",
    "FuelLtd",
    "FuelRate",
    "FuelTemperature",
    "Throttle",
    "TurboBoostPressure",
]
joined = joined[col_order]
joined.columns

Index(['EquipmentID', 'EventTimeStamp', 'spn', 'fmi', 'active',
       'derate_window', 'time_since_last_fault', 'fault_frequency', 'Latitude',
       'Longitude', 'nearStation', 'Speed', 'BarometricPressure',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLtd',
       'FuelRate', 'FuelTemperature', 'Throttle', 'TurboBoostPressure'],
      dtype='object')

In [10]:
display(joined.dtypes)

EquipmentID                         object
EventTimeStamp              datetime64[ns]
spn                                  int64
fmi                                  int64
active                                bool
derate_window                         bool
time_since_last_fault              float64
fault_frequency                      int64
Latitude                           float64
Longitude                          float64
nearStation                           bool
Speed                               object
BarometricPressure                  object
EngineCoolantTemperature            object
EngineLoad                          object
EngineOilPressure                   object
EngineOilTemperature                object
EngineRpm                           object
EngineTimeLtd                       object
FuelLtd                             object
FuelRate                            object
FuelTemperature                     object
Throttle                            object
TurboBoostP

In [11]:
comma_sub_cols = [
    'Speed',
'BarometricPressure',
'EngineCoolantTemperature',
'EngineLoad',
'EngineOilPressure',
'EngineOilTemperature',
'EngineRpm',
'FuelRate',
'FuelTemperature',
'Throttle',
'TurboBoostPressure',
'EngineTimeLtd',
'FuelLtd'
]

for col in comma_sub_cols:
    joined[col] = joined[col].str.replace(',', '.', regex=True)


In [12]:
dtypes = {
    'EquipmentID':object,
'EventTimeStamp':'datetime64[ns]',
'spn':int,
'fmi':int,
'active':bool,
'derate_window':bool,
'time_since_last_fault':float,
'fault_frequency':int,
'Latitude':float,
'Longitude':float,
'nearStation':bool,
'Speed':float,
'BarometricPressure':float,
'EngineCoolantTemperature':float,
'EngineLoad':float,
'EngineOilPressure':float,
'EngineOilTemperature':float,
'EngineRpm':float,
'EngineTimeLtd':float,
'FuelLtd':float,
'FuelRate':float,
'FuelTemperature':float,
'Throttle':float,
'TurboBoostPressure':float,
}
joined = joined.astype(dtype=dtypes)

In [13]:
for col in joined.columns:
    if joined[col].dtype == 'bool':
        joined[col] = joined[col].astype(int)
print(joined.dtypes)

EquipmentID                         object
EventTimeStamp              datetime64[ns]
spn                                  int64
fmi                                  int64
active                               int64
derate_window                        int64
time_since_last_fault              float64
fault_frequency                      int64
Latitude                           float64
Longitude                          float64
nearStation                          int64
Speed                              float64
BarometricPressure                 float64
EngineCoolantTemperature           float64
EngineLoad                         float64
EngineOilPressure                  float64
EngineOilTemperature               float64
EngineRpm                          float64
EngineTimeLtd                      float64
FuelLtd                            float64
FuelRate                           float64
FuelTemperature                    float64
Throttle                           float64
TurboBoostP

EquipmentID and EventTimeStamp will ultimately be dropped before being run through the model, they're just our grouping variables mostly. 

In [14]:
print(joined.isna().sum())
print(joined['Throttle'].value_counts())

EquipmentID                      0
EventTimeStamp                   0
spn                              0
fmi                              0
active                           0
derate_window                    0
time_since_last_fault         1108
fault_frequency                  0
Latitude                         0
Longitude                        0
nearStation                      0
Speed                        24538
BarometricPressure           22478
EngineCoolantTemperature     22383
EngineLoad                   22833
EngineOilPressure            22210
EngineOilTemperature         24542
EngineRpm                    21533
EngineTimeLtd                27088
FuelLtd                      23259
FuelRate                     23217
FuelTemperature             309344
Throttle                    187951
TurboBoostPressure           25103
dtype: int64
Throttle
100.0    286284
0.0      113423
38.4        226
38.8        226
37.6        225
          ...  
82.4         17
87.2         16
87.6      

In [15]:
for col in joined.columns:
    if joined[col].dtype == 'int64' or joined[col].dtype == 'float64':
        joined[col] = joined[col].bfill().ffill()

In [16]:
print(joined.isna().sum())

EquipmentID                 0
EventTimeStamp              0
spn                         0
fmi                         0
active                      0
derate_window               0
time_since_last_fault       0
fault_frequency             0
Latitude                    0
Longitude                   0
nearStation                 0
Speed                       0
BarometricPressure          0
EngineCoolantTemperature    0
EngineLoad                  0
EngineOilPressure           0
EngineOilTemperature        0
EngineRpm                   0
EngineTimeLtd               0
FuelLtd                     0
FuelRate                    0
FuelTemperature             0
Throttle                    0
TurboBoostPressure          0
dtype: int64


In [17]:
joined.to_csv('../data/window_2p0_model_data.csv')